In [ ]:
!pip install tensorflow
!pip install keras

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# To use the supplied dataset
X_train = np.load('mnist1_features_train.npy', allow_pickle=True)
y_train = np.load('mnist1_labels_train.npy', allow_pickle=True)
X_test = np.load('mnist1_features_test.npy', allow_pickle=True)
y_test = np.load('mnist1_labels_test.npy', allow_pickle=True)

In [2]:
# To use the full MNIST dataset
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# Reshapes the input in 3 dimensions (28, 28, 1) for each image and normalizes pixel values to be between 0 and 1.
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255

# Converts labels to categorical format (one-hot vectors).
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))

# Define Optimizer
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08)

# Compile model
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

# Adjusting learning rate
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# Adjusting epochs and batch_size
epochs = 50
batch_size = 128

# Data Augmentation 
datagen = ImageDataGenerator(
        featurewise_center=False, # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

train_gen = datagen.flow(X_train, y_train, batch_size=batch_size)

# Prediction model
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    validation_data=(X_test, y_test),
                    verbose=2,
                    batch_size=batch_size,
                    callbacks=[learning_rate_reduction])

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

Epoch 1/50


/Users/mael/.pyenv/versions/3.12.2/envs/default/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


469/469 - 128s - 273ms/step - accuracy: 0.9630 - loss: 0.1202 - val_accuracy: 0.8485 - val_loss: 0.5169 - learning_rate: 0.0010
Epoch 2/50
469/469 - 132s - 282ms/step - accuracy: 0.9868 - loss: 0.0430 - val_accuracy: 0.9900 - val_loss: 0.0313 - learning_rate: 0.0010
Epoch 3/50
469/469 - 135s - 288ms/step - accuracy: 0.9905 - loss: 0.0304 - val_accuracy: 0.9899 - val_loss: 0.0301 - learning_rate: 0.0010
Epoch 4/50
469/469 - 140s - 298ms/step - accuracy: 0.9918 - loss: 0.0261 - val_accuracy: 0.9911 - val_loss: 0.0274 - learning_rate: 0.0010
Epoch 5/50
469/469 - 132s - 282ms/step - accuracy: 0.9932 - loss: 0.0217 - val_accuracy: 0.9945 - val_loss: 0.0194 - learning_rate: 0.0010
Epoch 6/50
469/469 - 132s - 282ms/step - accuracy: 0.9945 - loss: 0.0181 - val_accuracy: 0.9943 - val_loss: 0.0216 - learning_rate: 0.0010
Epoch 7/50
469/469 - 133s - 284ms/step - accuracy: 0.9945 - loss: 0.0170 - val_accuracy: 0.9934 - val_loss: 0.0216 - learning_rate: 0.0010
Epoch 8/50

Epoch 8: ReduceLROnPlateau

In [4]:
print(f"Final Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Final Test Accuracy: {test_accuracy * 100:.2f}%")

Final Training Accuracy: 100.00%
Final Test Accuracy: 99.61%
